In [118]:
# libraries
import pandas as pd
import numpy as np
import sqlite3
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [119]:
# open the database(db).
#db = sqlite3.connect("../datasets/database.sqlite")

# convert database to dataframe.
#df = pd.read_sql_query("SELECT * FROM consumer_complaints", db)
#df

# About Dataset

Each week the CFPB sends thousands of consumers’ complaints about financial products and services to companies for response. Those complaints are published here after the company responds or after 15 days, whichever comes first. By adding their voice, consumers help improve the financial marketplace.

In [120]:
# loading dataset.
df = pd.read_csv('../datasets/consumer_complaints.csv', low_memory=False)
df.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [121]:
shape_rows = df.shape[0]
shape_columns = df.shape[1]

print(f'Dataset rows = {shape_rows} and columns = {shape_columns}.')

Dataset rows = 555957 and columns = 18.


In [122]:
df.dtypes

date_received                   object
product                         object
sub_product                     object
issue                           object
sub_issue                       object
consumer_complaint_narrative    object
company_public_response         object
company                         object
state                           object
zipcode                         object
tags                            object
consumer_consent_provided       object
submitted_via                   object
date_sent_to_company            object
company_response_to_consumer    object
timely_response                 object
consumer_disputed?              object
complaint_id                     int64
dtype: object

In [123]:
df.isna().sum()

date_received                        0
product                              0
sub_product                     158322
issue                                0
sub_issue                       343335
consumer_complaint_narrative    489151
company_public_response         470833
company                              0
state                             4887
zipcode                           4505
tags                            477998
consumer_consent_provided       432499
submitted_via                        0
date_sent_to_company                 0
company_response_to_consumer         0
timely_response                      0
consumer_disputed?                   0
complaint_id                         0
dtype: int64

In [124]:
# drop line with NaN values.
#df.dropna(inplace=True)

# For categorical data, you can replace missing values 
# with the mode (most frequent value) of the column.
df.fillna(df.mode().iloc[0], inplace=True) #FIXME Good for now, but you to check later if make sense.


shape_rows_cleaned = df.shape[0]
shape_columns_cleaned = df.shape[1]

print(f'before drop - Dataset rows = {shape_rows} and columns = {shape_columns}.')
print(f'after drop - Dataset rows = {shape_rows_cleaned} and columns = {shape_columns_cleaned}.')

before drop - Dataset rows = 555957 and columns = 18.
after drop - Dataset rows = 555957 and columns = 18.


In [125]:
df.isna().sum()

date_received                   0
product                         0
sub_product                     0
issue                           0
sub_issue                       0
consumer_complaint_narrative    0
company_public_response         0
company                         0
state                           0
zipcode                         0
tags                            0
consumer_consent_provided       0
submitted_via                   0
date_sent_to_company            0
company_response_to_consumer    0
timely_response                 0
consumer_disputed?              0
complaint_id                    0
dtype: int64

In [126]:
df.dtypes

date_received                   object
product                         object
sub_product                     object
issue                           object
sub_issue                       object
consumer_complaint_narrative    object
company_public_response         object
company                         object
state                           object
zipcode                         object
tags                            object
consumer_consent_provided       object
submitted_via                   object
date_sent_to_company            object
company_response_to_consumer    object
timely_response                 object
consumer_disputed?              object
complaint_id                     int64
dtype: object

In [127]:
df.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",Account status,This company continues to report on my credit ...,Company chooses not to provide a public response,U.S. Bancorp,CA,95993,Older American,Consent provided,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",Account status,This company continues to report on my credit ...,Company chooses not to provide a public response,Wells Fargo & Company,CA,91104,Older American,Consent provided,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,Other mortgage,Incorrect information on credit report,Account status,This company continues to report on my credit ...,Company chooses not to provide a public response,Wells Fargo & Company,NY,11764,Older American,Consent provided,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,This company continues to report on my credit ...,Company chooses not to provide a public response,"Navient Solutions, Inc.",MD,21402,Older American,Consent provided,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,This company continues to report on my credit ...,Company chooses not to provide a public response,Resurgent Capital Services L.P.,GA,30106,Older American,Consent provided,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [128]:
# Loop through all columns and convert 'object' columns to category
for column in df.columns:
    if column == 'date_received' or column == 'date_sent_to_company':
        df[column] = df[column].astype('datetime64[s]')
    
    else:
        df[column] = df[column].astype('category')

In [129]:
df.dtypes

date_received                   datetime64[ns]
product                               category
sub_product                           category
issue                                 category
sub_issue                             category
consumer_complaint_narrative          category
company_public_response               category
company                               category
state                                 category
zipcode                               category
tags                                  category
consumer_consent_provided             category
submitted_via                         category
date_sent_to_company            datetime64[ns]
company_response_to_consumer          category
timely_response                       category
consumer_disputed?                    category
complaint_id                          category
dtype: object

In [130]:
# category to encode.
label_encoder = LabelEncoder()

for column in df.columns:
    if df[column].dtype == 'category':
        df[column] = label_encoder.fit_transform(df[column])

df


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,2013-08-30,6,30,59,0,53801,7,3350,8,25820,0,1,4,2013-09-03,1,1,1,145498
1,2013-08-30,6,30,60,0,53801,7,3520,8,23991,0,1,4,2013-09-03,1,1,1,145499
2,2013-08-30,3,30,52,0,53801,7,3520,42,3302,0,1,3,2013-09-18,1,1,0,145331
3,2013-08-30,10,27,80,55,53801,7,2333,26,6172,0,1,0,2013-08-30,1,1,1,145259
4,2013-08-30,4,7,43,4,53801,7,2813,15,8725,0,1,5,2013-08-30,1,1,1,145495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555952,2014-07-01,6,30,59,0,53801,7,444,8,8719,0,1,4,2014-07-07,1,1,0,261909
555953,2014-07-01,6,30,60,0,53801,7,2799,8,8719,1,1,4,2014-07-23,1,0,0,261629
555954,2012-07-10,6,6,59,0,53801,7,416,8,8719,0,1,2,2013-11-18,1,1,0,38388
555955,2015-04-14,4,19,25,25,53801,7,1494,8,8719,0,1,2,2015-04-14,7,0,0,382328


In [131]:

datetimes = ['date_received', 'date_sent_to_company']

for idatetime in datetimes:
    df[f'{idatetime}'] = label_encoder.fit_transform(df[f'{idatetime}'])

In [132]:
# Instantiate the scalers
standard_scaler = StandardScaler()

In [133]:
# Use StandardScaler
data_scaler = standard_scaler.fit_transform(df)

data = pd.DataFrame(data_scaler,columns=df.columns) # nada a fazer aqui, aqui a linha de código está pronta
data

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,-0.710908,0.890282,0.690842,0.538056,-0.594246,0.26568,0.112937,1.423656,-1.198704,1.476636,-0.243840,0.228072,-0.395456,-0.724345,-0.458534,0.161007,1.989465,-0.825467
1,-0.710908,0.890282,0.690842,0.582816,-0.594246,0.26568,0.112937,1.579136,-1.198704,1.252757,-0.243840,0.228072,-0.395456,-0.724345,-0.458534,0.161007,1.989465,-0.825461
2,-0.710908,-0.402487,0.690842,0.224735,-0.594246,0.26568,0.112937,1.579136,0.757098,-1.279688,-0.243840,0.228072,-1.400266,-0.693296,-0.458534,0.161007,-0.502648,-0.826508
3,-0.710908,2.613973,0.431387,1.478021,2.426861,0.26568,0.112937,0.493520,-0.163279,-0.928385,-0.243840,0.228072,-4.414696,-0.733899,-0.458534,0.161007,1.989465,-0.826956
4,-0.710908,0.028436,-1.298310,-0.178108,-0.374529,0.26568,0.112937,0.932522,-0.796038,-0.615884,-0.243840,0.228072,0.609354,-0.733899,-0.458534,0.161007,1.989465,-0.825486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555952,0.006986,0.890282,0.690842,0.538056,-0.594246,0.26568,0.112937,-1.234137,-1.198704,-0.616618,-0.243840,0.228072,-0.395456,0.004129,-0.458534,0.161007,-0.502648,-0.100124
555953,0.006986,0.890282,0.690842,0.582816,-0.594246,0.26568,0.112937,0.919718,-1.198704,-0.616618,2.052426,0.228072,-0.395456,0.042344,-0.458534,-6.210920,-0.502648,-0.101869
555954,-1.690068,0.890282,-1.384795,0.538056,-0.594246,0.26568,0.112937,-1.259745,-1.198704,-0.616618,-0.243840,0.228072,-2.405076,-0.547601,-0.458534,0.161007,-0.502648,-1.492857
555955,0.682513,0.028436,-0.260492,-0.983792,0.778984,0.26568,0.112937,-0.273819,-1.198704,-0.616618,-0.243840,0.228072,-2.405076,0.675280,4.844366,-6.210920,-0.502648,0.650192
